In [48]:
import requests
import json
def get_current_contract_ticker():
    url = "https://api.elections.kalshi.com/trade-api/v2/events?status=open&series_ticker=KXBTC"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    data = response.json()

    # sort events by strike date
    data['events'].sort(key=lambda x: x['strike_date'])

    for event in data['events']:
        print(f"Event: {event['event_ticker']} Strike Date: {event['strike_date']}")

    # take first ticker 
    first_event = data['events'][0]
    print(f"First Event Ticker: {first_event['event_ticker']}")
    return first_event['event_ticker']

In [49]:
event = get_current_contract_ticker()

Event: KXBTC-25MAY2914 Strike Date: 2025-05-29T18:00:00Z
Event: KXBTC-25MAY2917 Strike Date: 2025-05-29T21:00:00Z
First Event Ticker: KXBTC-25MAY2914


In [ ]:
def get_options_chain_for_event(event, brti_price=0, threshold=1000):
    try:
        url = f"https://api.elections.kalshi.com/trade-api/v2/events/{event}"
        headers = {"accept": "application/json"}
        response = requests.get(url, headers=headers)

        chain = []
        res = json.loads(response.text)

        print(res['markets'])

        for m in res['markets']:
            things = m['suptitle'].split(" - ")

            bottom = int(things[0][1:])
            top = int(things[1][1:])

            print(m['subtitle'].strip(" "))

        for m in res['markets']:
            strike = (round(m['floor_strike'], 0))

            if abs(strike - brti_price) < threshold:
                chain.append(m)

        return chain    
    except Exception as e:
        print("❌ Error fetching chain:", e)
        return None, None

In [58]:
print(event)

KXBTC-25MAY2914


In [59]:
chain_data = get_options_chain_for_event(event, 106368, threshold=250)

[{'ticker': 'KXBTC-25MAY2914-T100000', 'event_ticker': 'KXBTC-25MAY2914', 'market_type': 'binary', 'title': '', 'subtitle': '$99,999.99 or below', 'yes_sub_title': '$99,999.99 or below', 'no_sub_title': '$99,999.99 or below', 'open_time': '2025-05-29T17:00:00Z', 'close_time': '2025-05-29T18:00:00Z', 'expected_expiration_time': '2025-05-29T18:05:00Z', 'expiration_time': '2025-06-05T18:00:00Z', 'latest_expiration_time': '2025-06-05T18:00:00Z', 'settlement_timer_seconds': 60, 'status': 'active', 'response_price_units': 'usd_cent', 'notional_value': 100, 'tick_size': 1, 'yes_bid': 0, 'yes_ask': 4, 'no_bid': 96, 'no_ask': 100, 'last_price': 0, 'previous_yes_bid': 0, 'previous_yes_ask': 0, 'previous_price': 0, 'volume': 0, 'volume_24h': 0, 'liquidity': 10157600, 'open_interest': 0, 'result': '', 'can_close_early': True, 'expiration_value': '', 'category': '', 'risk_limit_cents': 0, 'strike_type': 'less', 'cap_strike': 100000, 'rules_primary': "If the simple average of the sixty seconds of CF